# 自然语言和单词的分布式表示
- 编程语言是一种“硬语言”，自然语言是一种“软语言”
- 单词的含义：语言是由文字构成的，单词是文字的一部分，单词的含义是文字的意义。所以让计算机去理解单词的含义才是重点，让计算机理解单词的含义分为以下三种方法：
  - 基于同义词的字典方法
  - 基于计数的方法
  - 基于推理的方法
## 基于同义词的字典方法
像做新华字典那样：人为的设计同义词、上位-下位、整体-部分。知名的同义词字典方法：WordNet。使用同义词字典存在的问题：
- 难以顺应时代的变化
- 主要靠人，后期人力维护成本高